# Practical Exercise 4.03: Bullet, Barbell, Ladder Portfolios

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Upload the Excel file provided by the user

file_path = 'Set of bonds.xlsx'
df_bonds = pd.read_excel(file_path)

# Asking for bonds for the bullet portfolio

bullet_bonds_input = input("Enter the bond names for the bullet portfolio, separated by commas. (e.g. 'Bond D, Bond E'): ")

# Convert user input into a bond list

bullet_bonds = [bond.strip() for bond in bullet_bonds_input.split(',')]

# Make sure that the bonds entered exist in the data.

for bond in bullet_bonds:
    if bond not in df_bonds['Bond'].values:
        raise ValueError(f"Bond {bond} not found in the bond table.")

# Asking for bonds for the barbell portfolio

barbell_bonds_input = input("Enter the bond names for the barbell portfolio, separated by commas. (e.g. 'Bond A, Bond B, Bond C'): ")

# Convert user input into a bond list

barbell_bonds = [bond.strip() for bond in barbell_bonds_input.split(',')]

# Make sure that the bonds entered exist in the data.

for bond in barbell_bonds:
    if bond not in df_bonds['Bond'].values:
        raise ValueError(f"Bond {bond} not found in the bond table.")

# Asking for bonds for the ladder portfolio

ladder_bonds_input = input("Enter the bond names for the ladder portfolio, separated by commas. (e.g. 'Bond A, Bond C, Bond G'): ")

# Convert user input into a bond list

ladder_bonds = [bond.strip() for bond in ladder_bonds_input.split(',')]

# Make sure that the bonds entered exist in the data.

for bond in ladder_bonds:
    if bond not in df_bonds['Bond'].values:
        raise ValueError(f"Bond {bond} not found in the bond table.")



# Request total investment in euros

total_investment = float(input("Enter the total investment in euros (e.g. 1000000): "))


In [ ]:
# --------- Bullet Portfolio Calculation ---------

bullet_selected_bonds = df_bonds[df_bonds['Bond'].isin(bullet_bonds)]
num_bonds_bullet = len(bullet_selected_bonds)

# Calculate equal weight for each bond in the bullet portfolio

equal_weight = 1 / num_bonds_bullet

# Calculate the investment in euros for each bond in the bullet portfolio

investment_bullet = np.full(num_bonds_bullet, equal_weight) * total_investment

# Always round up

securities_bullet = np.ceil(investment_bullet / bullet_selected_bonds['Price'].values)

# Calculate Final Investment

final_investment_bullet = securities_bullet * bullet_selected_bonds['Price'].values

table_results_bullet = pd.DataFrame({
    'Bond': bullet_selected_bonds['Bond'].values,
    'Weight (%)': [equal_weight * 100] * num_bonds_bullet, # Equal weighting for all bonds
    'Investment €': investment_bullet,
    'Price €': bullet_selected_bonds['Price'].values,
    'Securities': securities_bullet,
    'Final Investment €': final_investment_bullet,
    'YTM (%)': bullet_selected_bonds['YTM (%)'].values * 100,
    'Modified Duration': bullet_selected_bonds['Modified Duration'].values,
    'Convexity': bullet_selected_bonds['Convexity'].values
})

# Calculate the aggregate characteristics of the bullet portfolio

bullet_portfolio_ytm = np.average(bullet_selected_bonds['YTM (%)'], weights=investment_bullet)
bullet_portfolio_convexity = np.average(bullet_selected_bonds['Convexity'], weights=investment_bullet)
bullet_portfolio_modified_duration = np.average(bullet_selected_bonds['Modified Duration'], weights=investment_bullet)

row_bullet_portfolio = pd.DataFrame({
    'Bond': ['Bullet Portfolio'],  # Summary row
    'Weight (%)': [100.0],
    'Investment €': [investment_bullet.sum()],
    'Price €': [np.nan],  # Leave empty to not apply formatting
    'Securities': [np.nan],  # Leave empty to not apply formatting
    'Final Investment €': [final_investment_bullet.sum()],
    'YTM (%)': [bullet_portfolio_ytm * 100],
    'Modified Duration': [bullet_portfolio_modified_duration],
    'Convexity': [bullet_portfolio_convexity]
})


In [ ]:
# --------- Barbell Portfolio Calculation ---------

selected_bonds = df_bonds[df_bonds['Bond'].isin(barbell_bonds)]
modified_durations = selected_bonds['Modified Duration'].values

def target(weights):
    portfolio_modified_duration = np.dot(weights, modified_durations)
    return (portfolio_modified_duration - bullet_portfolio_modified_duration) ** 2

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = [(0, 1) for _ in range(len(modified_durations))]
initial_weights = np.full(len(modified_durations), 1 / len(modified_durations))

results = minimize(target, initial_weights, bounds=bounds, constraints=constraints)
optimal_weights = results.x

investment_barbell = optimal_weights * total_investment

# Always round up

securities_barbell = np.ceil(investment_barbell / selected_bonds['Price'].values)
# Calculate Final Investment
final_investment_barbell = securities_barbell * selected_bonds['Price'].values

ytm_optimal_portfolio = np.dot(optimal_weights, selected_bonds['YTM (%)'].values)
convexity_optimal_portfolio = np.dot(optimal_weights, selected_bonds['Convexity'].values)
modified_duration_optimal_portfolio = np.dot(optimal_weights, selected_bonds['Modified Duration'].values)

table_results_barbell = pd.DataFrame({
    'Bond': selected_bonds['Bond'].values,
    'Weight (%)': optimal_weights * 100,  # Convert weights to %
    'Investment €': investment_barbell,
    'Price €': selected_bonds['Price'].values,
    'Securities': securities_barbell,
    'Final Investment €': final_investment_barbell,
    'YTM (%)': selected_bonds['YTM (%)'].values * 100,  # Convert YTM to %
    'Modified Duration': selected_bonds['Modified Duration'].values,
    'Convexity': selected_bonds['Convexity'].values
})

row_barbell_portfolio = pd.DataFrame({
    'Bond': ['Barbell Portfolio'],
    'Weight (%)': [100.0],
    'Investment €': [investment_barbell.sum()],
    'Price €': [np.nan],  # Leave empty to not apply formatting
    'Securities': [np.nan],  # Leave empty to not apply formatting
    'Final Investment €': [final_investment_barbell.sum()],
    'YTM (%)': [ytm_optimal_portfolio * 100],  # Convert YTM to %
    'Modified Duration': [modified_duration_optimal_portfolio],
    'Convexity': [convexity_optimal_portfolio]
})


In [ ]:
# --------- Ladder Portfolio Calculation ---------

ladder_selected_bonds = df_bonds[df_bonds['Bond'].isin(ladder_bonds)]
modified_durations_ladder = ladder_selected_bonds['Modified Duration'].values

# Optimization to match ladder portfolio's modified duration with bullet portfolio's

def ladder_target(weights):
    portfolio_modified_duration = np.dot(weights, modified_durations_ladder)
    return (portfolio_modified_duration - bullet_portfolio_modified_duration) ** 2

constraints_ladder = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds_ladder = [(0, 1) for _ in range(len(modified_durations_ladder))]
initial_weights_ladder = np.full(len(modified_durations_ladder), 1 / len(modified_durations_ladder))

results_ladder = minimize(ladder_target, initial_weights_ladder, bounds=bounds_ladder, constraints=constraints_ladder)
optimal_weights_ladder = results_ladder.x

investment_ladder = optimal_weights_ladder * total_investment

# Always round up

securities_ladder = np.ceil(investment_ladder / ladder_selected_bonds['Price'].values)

# Calculate Final Investment

final_investment_ladder = securities_ladder * ladder_selected_bonds['Price'].values

ytm_ladder_portfolio = np.dot(optimal_weights_ladder, ladder_selected_bonds['YTM (%)'].values)
convexity_ladder_portfolio = np.dot(optimal_weights_ladder, ladder_selected_bonds['Convexity'].values)
modified_duration_ladder_portfolio = np.dot(optimal_weights_ladder, ladder_selected_bonds['Modified Duration'].values)

table_results_ladder = pd.DataFrame({
    'Bond': ladder_selected_bonds['Bond'].values,
    'Weight (%)': optimal_weights_ladder * 100,  # Convert to %
    'Investment €': investment_ladder,
    'Price €': ladder_selected_bonds['Price'].values,
    'Securities': securities_ladder,
    'Final Investment €': final_investment_ladder,
    'YTM (%)': ladder_selected_bonds['YTM (%)'].values * 100,  # Convert YTM to %
    'Modified Duration': ladder_selected_bonds['Modified Duration'].values,
    'Convexity': ladder_selected_bonds['Convexity'].values
})

row_ladder_portfolio = pd.DataFrame({
    'Bond': ['Ladder Portfolio'],
    'Weight (%)': [100.0],
    'Investment €': [investment_ladder.sum()],
    'Price €': [np.nan],  # Leave empty to not apply formatting
    'Securities': [np.nan],  # Leave empty to not apply formatting
    'Final Investment €': [final_investment_ladder.sum()],
    'YTM (%)': [ytm_ladder_portfolio * 100],
    'Modified Duration': [modified_duration_ladder_portfolio],
    'Convexity': [convexity_ladder_portfolio]
})


In [ ]:
# --------- Concatenate All Results ---------

table_results = pd.concat([
    table_results_bullet, row_bullet_portfolio,
    table_results_barbell, row_barbell_portfolio,
    table_results_ladder, row_ladder_portfolio
], ignore_index=True)

# Format all columns to display values to two decimal places, excluding NaN cells

table_results['Weight (%)'] = table_results['Weight (%)'].map('{:.2f}%'.format)
table_results['Investment €'] = table_results['Investment €'].map('{:,.2f}'.format)
table_results['Price €'] = table_results['Price €'].map(lambda x: '{:,.2f}'.format(x) if pd.notnull(x) else '')
table_results['Securities'] = table_results['Securities'].map(lambda x: '{:,.0f}'.format(x) if pd.notnull(x) else '')
table_results['Final Investment €'] = table_results['Final Investment €'].map('{:,.2f}'.format)
table_results['YTM (%)'] = table_results['YTM (%)'].map('{:.2f}%'.format)
table_results['Modified Duration'] = table_results['Modified Duration'].map('{:.2f}'.format)
table_results['Convexity'] = table_results['Convexity'].map('{:.2f}'.format)

# Show complete table

table_results
